In [2]:
# imports
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim
 
geolocator = Nominatim(user_agent="geoapiExercises")

In [3]:
scale = 'scale_1x/'
directory = 'fragments/'
filename = 'Fr_main_L3.csv'
table = pd.read_csv('Data/' +scale+directory+filename)
table


,ozone,particullate_matter,carbon_monoxide,sulfure_dioxide,nitrogen_dioxide,longitude,latitude,timestamp
0,64,78,34,47,24,10.124126,56.085585,2014-08-01 00:05:00
1,62,73,37,42,21,10.124126,56.085585,2014-08-01 00:10:00
2,65,76,40,43,17,10.124126,56.085585,2014-08-01 00:15:00
3,61,75,44,42,22,10.124126,56.085585,2014-08-01 00:20:00
4,57,78,40,47,19,10.124126,56.085585,2014-08-01 00:25:00
...,...,...,...,...,...,...,...,...
99996,140,33,158,64,144,10.167442,56.120041,2014-09-12 05:05:00
99997,144,29,156,63,145,10.167442,56.120041,2014-09-12 05:10:00
99998,139,25,160,59,140,10.167442,56.120041,2014-09-12 05:15:00
99999,137,29,156,64,138,10.167442,56.120041,2014-09-12 05:20:00


In [4]:
#get location 
table.iloc[0].longitude
loc = geolocator.geocode(str(table.iloc[0].latitude) +","+str(table.iloc[0].longitude))
print(loc)

Landevejen, Tranbjergparken, Tranbjerg, Aarhus Kommune, Region Midtjylland, 8310, Danmark


In [5]:
indexes = {
    "ozone": 1,
    "particullate_matter": 2,
    "carbon_monoxide": 3,
    "sulfure_dioxide": 4,
    "nitrogen_dioxide": 5,
    "longitude": 6,
    "latitude": 7,
    "timestamp": 8
}
indexes

{'ozone': 1,
 'particullate_matter': 2,
 'carbon_monoxide': 3,
 'sulfure_dioxide': 4,
 'nitrogen_dioxide': 5,
 'longitude': 6,
 'latitude': 7,
 'timestamp': 8}

In [6]:
GOOD_AQI = 49
AVERAGE_AQI = 149


good_AQI_count =0
average_AQI_count =0
poor_AQI_count =0

for tup in table.itertuples():
    if tup.ozone <= GOOD_AQI:
        good_AQI_count += 1
    elif tup.ozone > GOOD_AQI  and tup.ozone <= AVERAGE_AQI:
        average_AQI_count += 1
    else:
        poor_AQI_count += 1
print(good_AQI_count,average_AQI_count,poor_AQI_count)

13856 55178 30967


In [7]:
# sorting by columns
PM_table = table.copy()
PM_table = PM_table.sort_values(by=['particullate_matter'])

CO_table = table.copy()
CO_table = CO_table.sort_values(by=['carbon_monoxide'])

SO2_table = table.copy()
SO2_table = SO2_table.sort_values(by=['sulfure_dioxide'])

NO2_table = table.copy()
NO2_table = NO2_table.sort_values(by=['nitrogen_dioxide'])


In [8]:
AQI_ranges = [[0, 20], [21, 40], [41, 60], [61, 80], [81, 300]]

def getRange(attr_val):
    # print(attr_val)
    for r in AQI_ranges:
        if(int(attr_val) >= r[0] and int(attr_val) <= r[1]):
            return r

In [9]:
def isGood(val):
    if val <= GOOD_AQI:
        return True
    else:
        return False

def isAverage(val):
    if(val > GOOD_AQI and val <= AVERAGE_AQI):
        return True
    else:
        return False

def isPoor(val):
    if val > AVERAGE_AQI:
        return True
    else:
        return False



In [10]:
def checkComplete(attr_index,patterns):
    pattern_count = 0
    original_count = 0
    for tup in table.itertuples():
        if isGood(tup.ozone):
            original_count += 1
        for pattern in patterns:
            if tup[attr_index] >= pattern[0] and tup[attr_index] <= pattern[1]:
                pattern_count += 1
    print(pattern_count)
    print(original_count)
    if pattern_count == original_count:
        return True
    else:
        return False


In [11]:
def isPatternType(pattern_type , val):
    if pattern_type == 1:
        return isGood(val)
    elif pattern_type == 2:
        return isAverage(val)
    else:
        return isPoor(val)
    

In [12]:
def findPatterns_L1(attr, pattern_type):
   
    attr_table = table

    patterns = []
    last = ''
    prev = ''
    curr_count = 0
    curr_sum = 0

    summaries = {
        'particullate_matter': [],
        'carbon_monoxide': [],
        'sulfure_dioxide': [],
        'nitrogen_dioxide': [],
        's_timestamp': [],
        'e_timestamp': [],
        'count': [],
        'average': []
    }

    first = True
    
    for i in attr_table.itertuples():
       
        if(first):
            first = False
            last = i[indexes[attr]]

        if not isPatternType(pattern_type ,i.ozone):
            if curr_count > 0:
                patterns.append([last, prev, curr_count,(curr_sum/curr_count)])
            last = prev
            curr_count = 0
            curr_sum = 0
        else:
            curr_count += 1
            curr_sum += i.ozone
        prev = i[indexes[attr]]
    #print(patterns)
    patterns = sorted(patterns, key=lambda x: x[2], reverse=True)


    print("Number of patterns Founded : ", len(patterns))

    for pattern in patterns:
        summaries['particullate_matter'].append('*')
        summaries['carbon_monoxide'].append('*')
        summaries['sulfure_dioxide'].append('*')
        summaries['nitrogen_dioxide'].append('*')
        summaries['s_timestamp'].append(pattern[0])
        summaries['e_timestamp'].append(pattern[1])
        summaries['count'].append(pattern[2])
        summaries['average'].append(pattern[3])

    #print(summaries)
    df = pd.DataFrame(summaries)
    
    return df
good_summaries = findPatterns_L1('timestamp',1)
good_summaries.to_csv('Data/' +scale+ 'summaries/good_summaries_' + filename ,mode='w',index=False, header=False)

average_summaries = findPatterns_L1('timestamp',2)
average_summaries.to_csv('Data/' +scale+ 'summaries/average_summaries_' + filename ,mode='w',index= False,header=False)

poor_summaries = findPatterns_L1('timestamp',3)
poor_summaries.to_csv('Data/' +scale+ 'summaries/poor_summaries_' + filename ,mode='w',index = False,header=False)


Number of patterns Founded :  691
Number of patterns Founded :  1458
Number of patterns Founded :  768


In [13]:
def getPatterns():
    set1 = ['carbon_monoxide', 'sulfure_dioxide']
    patterns = []
    rangeGoodCountCO = [0, 0, 0, 0, 0]

    range = 200
    allMatchTup = []
    goodCount = 0
    count = 0
    for tuple in table.itertuples():
        # Good AQI for Ozone
        if(getRange(tuple.carbon_monoxide) == [81, 300]):
            count += 1
        if int(tuple.ozone) <= 99:
            goodCount += 1
            co_range = getRange(tuple.carbon_monoxide)
            so2_range = getRange(tuple.sulfure_dioxide)
            # print(tuple.carbon_monoxide)
            rangeGoodCountCO[AQI_ranges.index(
                getRange(tuple.carbon_monoxide))] += 1

            # for tup in table:
            #   if getRange() and getRange() :
            #     matchTups.append(tup)

    # patterns = []
    # return patterns
    print(goodCount)
    print("CO : ", count)
    print(rangeGoodCountCO)


getPatterns()


39987
CO :  64010
[445, 5297, 5026, 4748, 24471]


In [14]:
# power set calculation

def getPowerset(attributes, level):

    set = attributes
    set_size = len(set)

    pow_set_size = (int)(math.pow(2, set_size))
    counter = 0
    j = 0
    power_set = []

    for counter in range(0, pow_set_size):
        temp_set = []
        for j in range(0, set_size):
            if((counter & (1 << j)) > 0):
                temp_set.append(set[j])
        power_set.append(temp_set)

    print(power_set)


